In [1]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
import sys
sys.path.insert(0, "/Users/fultonwang/Documents/infembed")

Automatic pdb calling has been turned ON


define the cb model

In [2]:
import functools
from models._core.cb_decoder_llm import CBDecoderLightningModule, constructor
from models._utils.common import default_checkpoints_load_func, load_model


model = load_model(
    model=CBDecoderLightningModule(
        decoder=constructor(
            model_dim=768,
            key_dim=48,
            value_dim=48,
            num_heads=16,
            num_layers=4,
            dropout=0.0,
            hidden_dim=3072,
            num_tokens=10000,
            max_len=2048,
            num_concepts=4,
            concept_embedding_dim=32,
            concept_embedder_hidden_dims=None,
            concept_generator_hidden_dims=None,
            generator_hidden_dims=None,
        )
    ),
    eval=True,
    checkpoints_load_func=functools.partial(default_checkpoints_load_func, key='state_dict'),
    # checkpoint="/Users/fultonwang/Documents/infembed/examples/tinystories_cb/hydra_outputs/lightning_train/cb_simplified_read_julius_only_accum_2/lightning_logs/fqtzvlgh/checkpoints/epoch=5-step=1386.ckpt",
    checkpoint="/Users/fultonwang/Documents/infembed/examples/tinystories_cb/hydra_outputs/lightning_train/cb_simplified_read_julius_only_accum_2/lightning_logs/fqtzvlgh/checkpoints/epoch=3-step=924.ckpt",
    # checkpoint="/home/ubuntu/Documents/infembed/examples/tinystories_cb/hydra_outputs/lightning_train/cb_simplified_read_julius_only_accum_2/lightning_logs/fqtzvlgh/checkpoints/epoch=5-step=1386.ckpt",
)
model

/Users/fultonwang/anaconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


<All keys matched successfully>


CBDecoderLightningModule(
  (decoder): CBDecoder(
    (decoder_layers): ModuleList(
      (0-3): 4 x DecoderLayer(
        (attention): MultiAttention(
          (attentions): ModuleList(
            (0-15): 16 x Attention(
              (key): Linear(in_features=768, out_features=48, bias=True)
              (query): Linear(in_features=768, out_features=48, bias=True)
              (value): Linear(in_features=768, out_features=48, bias=True)
            )
          )
          (projection): Linear(in_features=768, out_features=768, bias=True)
        )
        (feedforward): FeedForward(
          (linear_1): Linear(in_features=768, out_features=3072, bias=True)
          (linear_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (attention_sublayer): Sublayer(
          (dropout): Dropout(p=0.0, inplace=False)
          (layer_norm): LayerNorm()
        )
        (feedforward_sublayer): Sublayer(
          (d

define the dataloader to get explanations and guide

In [3]:
from torch.utils.data import Dataset, DataLoader
from data._core.tinystories import tinystories_tokenizer
from data._utils.llm import DecoderLLMCollateFn
import pandas as pd


dataset_path = '/Users/fultonwang/Documents/infembed/files/tinystories/generations/wandb_export_2024-02-22T12_01_34.815-05_00.csv'
# dataset_path = '/home/ubuntu/Documents/infembed/files/tinystories/generations/wandb_export_2024-02-22T12_01_34.815-05_00.csv'

# define adhoc dataset
class GenerationDataset(Dataset):
    def __init__(self, path):
        self.df = pd.read_csv(path)

    def __getitem__(self, i):
        return self.df["generation"].iloc[i]

    def __len__(self):
        return len(self.df)

dataset = GenerationDataset(dataset_path)
dataloader = DataLoader(
    dataset=dataset,
    collate_fn=DecoderLLMCollateFn(
        tokenizer=tinystories_tokenizer(),
        max_len=512,
    ),
    batch_size=1,
)

next(iter(dataloader))

{'labels': tensor([[  41,  740, 2402,  259,  376,  477,   14,  338,  740, 2402,  259,  376,
           500,   14,  338, 1218, 1735, 5190,   14,  338,  964,  460, 1233,   76,
          1322, 5743]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]),
 'input_ids': tensor([[   0,   41,  740, 2402,  259,  376,  477,   14,  338,  740, 2402,  259,
           376,  500,   14,  338, 1218, 1735, 5190,   14,  338,  964,  460, 1233,
            76, 1322]]),
 'mask': tensor([[ True, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False],
         [ True,  True,  True, False, False, F

helpers

In [54]:
from data._core.tinystories import tinystories_tokenizer
from models._utils.binary_multitask_llm import _last_token_multitask_get_preds
import matplotlib.pyplot as plt
from IPython.core.display import HTML, display


tokenizer = tinystories_tokenizer()


def get_batch_tokens(input_ids, attention_mask):
    example_tokens = []
    for _input_ids, _attention_mask in zip(input_ids, attention_mask):
        _example_tokens = [
            tokenizer.decode(id)
            for (id, mask) in zip(_input_ids, _attention_mask)
            if mask == 1
        ]
        example_tokens.append(_example_tokens)

    return example_tokens


import torch


def get_batch_predictions(input_ids, attention_mask, mask):
    raw_concept_logits = model.forward({"input_ids": input_ids, "mask": mask})[
        "concept_logits"
    ]
    concept_logits = []
    for _concept_logits, _attention_mask in zip(raw_concept_logits, attention_mask):
        concept_logits.append(
            torch.stack([__concept_logits for (__concept_logits, mask) in zip(_concept_logits, _attention_mask) if mask == 1], dim=0)
        )
    return concept_logits


def plot_example_tokens_single_concept(_example_tokens, _concept_logits, c):
    """
    plots example, for the provided concept
    """
    print(
        " ".join(
            [
                f"({token}, {torch.sigmoid(logit): .2f})"
                for (token, logit) in zip(_example_tokens, _concept_logits[:, c])
            ]
        )
    )

def _plot_example_tokens_single_concept(_example_tokens, _concept_logits, c):
    attrs = (torch.sigmoid(_concept_logits[:, c]) - 0.5) * 2
    # attrs = torch.sigmoid(_concept_logits[:, c])
    # rgb = lambda x: '255,0,0' if x < 0 else '0,255,0'
    rgb = lambda x: '0,255,0' if x < 0 else '255,0,0'
    # rgb = lambda x: '255,0,0'
    alpha = lambda x: abs(x) ** 0.25
    token_marks = [
        f'<mark style="background-color:rgba({rgb(attr)},{alpha(attr)})">{token}</mark>'
        for token, attr in zip(_example_tokens[1:], attrs.tolist()[1:])
    ]
    
    display(HTML('<p>' + ' '.join(token_marks) + '</p>'))


def get_batch_example_predictions(concept_logits):
    return [_concept_logits[-1].detach().numpy() for _concept_logits in concept_logits]


def plot_example_all_concepts(_example_tokens, _example_concept_logits):
    print(''.join(_example_tokens[1:]))
    fig = plt.figure(figsize=(4,3))
    ax = fig.add_subplot(1,1,1)
    # fig, ax = plt.subplots()
    fig.suptitle('probability of different concepts', fontsize=20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_ylim((0,1))
    pd.Series(dict(zip(range(_example_concept_logits.shape[1]), torch.sigmoid(torch.Tensor(_example_concept_logits)).numpy()))).plot.bar(ax=ax)
    plt.show()

def print_tokens(_tokens):
    print(''.join(_tokens))

def print_ids(_ids):
    print_tokens(tokenizer.decode(_ids))

/var/folders/hb/tpn62f1149951598g16hpts00000gn/T/ipykernel_13704/3039558482.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


plot per-token concept predictions

In [37]:
cs = [1,2]

for batch, _ in zip(dataloader, range(2)):
    example_tokens = get_batch_tokens(batch['input_ids'], batch['attention_mask'])
    concept_logits = get_batch_predictions(batch['input_ids'], batch['attention_mask'], batch['mask'])
    for c in cs:
        print(f"\n ### concept {c} ###")
        for _example_tokens, _concept_logits in zip(example_tokens, concept_logits):
            plot_example_tokens_single_concept(_example_tokens, _concept_logits, c)


 ### concept 1 ###
(<|endoftext|>,  0.10) (I,  0.01) ( am,  0.01) ( such,  0.00) ( a,  0.06) ( happy,  0.33) ( girl,  0.92) (.,  0.97) ( I,  0.16) ( am,  0.85) ( such,  0.16) ( a,  0.93) ( happy,  0.91) ( man,  0.50) (.,  0.90) ( I,  0.32) ( enjoy,  0.30) ( eating,  0.30) ( oranges,  0.39) (.,  0.91) ( I,  0.23) ( strong,  0.57) (ly,  0.48) ( dis,  0.06) (l,  0.08) (ike,  0.11)

 ### concept 2 ###
(<|endoftext|>,  0.84) (I,  0.00) ( am,  0.00) ( such,  0.05) ( a,  0.07) ( happy,  0.18) ( girl,  0.05) (.,  0.04) ( I,  0.20) ( am,  0.36) ( such,  0.25) ( a,  0.76) ( happy,  0.75) ( man,  0.99) (.,  0.91) ( I,  0.84) ( enjoy,  0.73) ( eating,  0.30) ( oranges,  0.54) (.,  0.80) ( I,  0.63) ( strong,  0.94) (ly,  0.76) ( dis,  0.20) (l,  0.24) (ike,  0.32)

 ### concept 1 ###
(<|endoftext|>,  0.10) (I,  0.01) ( am,  0.01) ( such,  0.00) ( a,  0.06) ( happy,  0.33) ( girl,  0.92) (.,  0.97) ( I,  0.16) ( am,  0.85) ( such,  0.16) ( a,  0.93) ( happy,  0.91) ( man,  0.50) (.,  0.90) ( I,  0

compare one-pass generations to generations if starting with different prefixes

In [51]:
from models._utils.cb_llm import GreedyCBDecoder


decoder = GreedyCBDecoder(max_len=512)

input_id = torch.Tensor([0]).long()
oneshot = decoder(
    model,
    eos_token=0,
    input_ids=torch.Tensor([0]).long(),
    temperature=0,
)

for t in range(10):
    head = torch.cat([input_id, oneshot[:t]])
    # head = oneshot[:t+1]

    tail = decoder(
        model,
        eos_token=0,
        input_ids=head,
        # input_ids=input_id,
        temperature=0,
    )
    print(tokenizer.decode(torch.cat([head, tail])))

<|endoftext|>I am such a happy girl. I am such a happy man. I enjoy eating oranges. I strongly dislike poems! Once upon a time there was a little girl named Lily. She was very curious and wanted to explore the world.
<|endoftext|>I am such a happy girl. I am such a happy man. I enjoy eating oranges. I strongly dislike poems! Once upon a time there was a little girl named Lily. She was very curious and wanted to explore the world.
<|endoftext|>I am such a happy girl. I am such a happy man. I enjoy eating oranges. I strongly dislike poems! Once upon a time there was a little girl named Lily. She was very curious and wanted to explore the world.
<|endoftext|>I am such a happy girl. I am such a happy man. I enjoy eating oranges. I strongly dislike poems! Once upon a time there was a little girl named Lily. She was very curious and wanted to explore the world.
<|endoftext|>I am such a happy girl. I am such a happy man. I enjoy eating oranges. I strongly dislike poems! Once upon a time there

define how to decide when to start generating on the basis of tokens and predictions

In [61]:
def _decider(threshold, c, prep_steps, _example_tokens, _concept_logits):
    return torch.max((torch.sigmoid(_concept_logits[:, c]) > threshold).float().argmax() - prep_steps, 0).values

decider = functools.partial(_decider, 0.90, 1, 0)

for each example, regenerate

In [66]:
from models._utils.cb_llm import ConstantStrategy


decoder = GreedyCBDecoder(max_len=512, strategy=ConstantStrategy([-1, 0, -1, -1]))


for batch, _ in zip(dataloader, range(2)):
    input_ids, attention_mask = batch['input_ids'], batch['attention_mask']
    concept_logits = model.decoder.full_generate(input_ids, attention_mask)['concept_logits']
    for (_input_ids, _attention_mask, _concept_logits) in zip(input_ids, attention_mask, concept_logits):
        print_ids(_input_ids)
        _example_tokens = tokenizer.decode(_input_ids)
        decider_pos = decider(_example_tokens, _concept_logits)
        head = _input_ids[:decider_pos]
        print_ids(head)
        tail = decoder(model=model, eos_token=0, input_ids=head)
        print_ids(torch.cat([head, tail]))
        import pdb
        pdb.set_trace()
        print()

<|endoftext|>I am such a happy girl. I am such a happy man. I enjoy eating oranges. I strongly dislike
<|endoftext|>I am such
<|endoftext|>I am such a happy girl. I am such a happy boy. I enjoy eating oranges. I am such a happy girl. I am such a happy girl.
> /var/folders/hb/tpn62f1149951598g16hpts00000gn/T/ipykernel_13704/55198528.py(20)<module>()
     16         tail = decoder(model=model, eos_token=0, input_ids=head)
     17         print_ids(torch.cat([head, tail]))
     18         import pdb
     19         pdb.set_trace()
---> 20         print()

tensor([[ 1.5954,  0.8001,  1.4446,  0.2316],
        [-1.1748, -0.6768,  0.5953, -2.2600],
        [ 1.6213, -0.1765,  1.6003,  0.8364],
        [ 0.0236,  0.3453,  1.6015, -0.7578],
        [ 0.9666,  1.9228,  2.3193, -1.8900],
        [ 0.4621,  1.4308,  3.7502, -1.0188],
        [ 0.2215,  3.8125,  1.7750,  0.8181],
        [ 1.4771,  2.6986,  2.4015,  0.5246],
        [-1.0237, -0.9713,  0.4538, -2.5780],
        [ 1.6344, -0.4428, 

define how to regenerate

In [21]:
from models._utils.cb_llm import ConstantStrategy, GreedyCBDecoder


decoder = GreedyCBDecoder(max_len=512, strategy=ConstantStrategy([-1, 0, -1, -1]))

for each example, shorten using decider, print original, shortened, new

In [28]:
from data._utils.llm import subsequent_mask
import torch.nn.functional as F

num_batches = 1


def get_batch_input_ids(batch):
    # returns list of tensors
    return [
        torch.Tensor([id for (id, mask) in zip(ids, attention_mask) if mask == 1]).long()
        for (ids, attention_mask) in zip(batch["input_ids"], batch["attention_mask"])
    ]


for batch, _ in zip(dataloader, range(num_batches)):
    example_tokens = get_batch_tokens(batch)
    __concept_logits = get_batch_predictions(batch)[the_concept]
    input_ids = get_batch_input_ids(batch)
    for _example_tokens, _concept_logits, _input_ids in zip(
        example_tokens, __concept_logits, input_ids
    ):
        decider_pos = decider(_example_tokens, _concept_logits)
        print("decider_pos", decider_pos)
        print("decider token", tokenizer.decode(_input_ids[decider_pos]))
        print("### original ###")
        plot([_example_tokens], [_concept_logits])
        print("### dangerous prefix")
        plot([_example_tokens[: decider_pos + 1]], [_concept_logits[: decider_pos + 1]])
        # generate starting with the last position in the prefix
        # initial_pos = decider_pos
        initial_pos = None
        _output_ids = decoder(model, eos_token=1, input_ids=_input_ids[:decider_pos + 1], initial_pos=initial_pos, temperature=None)#temperature=0.5)
        # get new ids
        # _new_ids = torch.cat([_input_ids[:-1], _output_ids])
        _new_ids = torch.cat([_input_ids[:decider_pos + 0], _output_ids])
        #_new_ids = _output_ids
        _new_attention_mask = torch.ones(len(_new_ids))
        # get corresponding tokens (batch format)
        new_batch = {
            "input_ids": _new_ids.unsqueeze(0),
            "attention_mask": _new_attention_mask.unsqueeze(0),
            "mask": subsequent_mask(len(_new_ids))
        }
        new_example_tokens = get_batch_tokens(new_batch)
        # feed through model to get concept predictions
        new_concept_logits = get_batch_predictions(new_batch)[the_concept]
        print("### new ###")
        # print(len(_new_ids), len(_input_ids[:-1]), len(_output_ids))
        # print(_new_ids)
        # print(new_example_tokens)
        plot(new_example_tokens, new_concept_logits)

        if False:
            _new_ids = _output_ids
            _new_attention_mask = torch.ones(len(_new_ids))
            # get corresponding tokens (batch format)
            new_batch = {
                "input_ids": _new_ids.unsqueeze(0),
                "attention_mask": _new_attention_mask.unsqueeze(0),
            }
            new_example_tokens = get_batch_tokens(new_batch)
            # feed through model to get concept predictions
            new_concept_logits = get_batch_predictions(batch)[the_concept]
            print("### new ###")
            print(len(_new_ids), len(_output_ids))
            print(_new_ids)
            print(new_example_tokens)
            plot(new_example_tokens, new_concept_logits)

        # also look at the top next tokens
        print()
        import pdb
        #pdb.set_trace()
        next_probs = F.softmax(decoder.next_logits(model, _input_ids[:decider_pos + 1], initial_pos=decider_pos))
        num_to_display = 10
        top_ids = next_probs.argsort(descending=True)[:num_to_display]
        top_tokens = get_batch_tokens({'input_ids': top_ids.unsqueeze(0), 'attention_mask': torch.ones(len(top_ids)).unsqueeze(0)})[0]
        top_probs = next_probs[top_ids]
        print(tokenizer.decode(top_ids))
        print(list(zip(top_ids, top_tokens, top_probs)))
        import pdb
        pdb.set_trace()
        plot([top_tokens], [top_probs])
        #print(list(zip(top_tokens, top_probs)))


decider_pos tensor(6)
decider token  girl
### original ###
(<|endoftext|>,  0.10) (I,  0.01) ( am,  0.01) ( such,  0.00) ( a,  0.06) ( happy,  0.33) ( girl,  0.92) (.,  0.97) ( I,  0.16) ( am,  0.85) ( such,  0.16) ( a,  0.93) ( happy,  0.91) ( man,  0.50) (.,  0.90) ( I,  0.32) ( enjoy,  0.30) ( eating,  0.30) ( oranges,  0.39) (.,  0.91) ( I,  0.23) ( strong,  0.57) (ly,  0.48) ( dis,  0.06) (l,  0.08) (ike,  0.11)
### dangerous prefix
(<|endoftext|>,  0.10) (I,  0.01) ( am,  0.01) ( such,  0.00) ( a,  0.06) ( happy,  0.33) ( girl,  0.92)


/var/folders/hb/tpn62f1149951598g16hpts00000gn/T/ipykernel_13704/1564019124.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  next_probs = F.softmax(decoder.next_logits(model, _input_ids[:decider_pos + 1], initial_pos=decider_pos))


### new ###
(<|endoftext|>,  0.10) (I,  0.01) ( am,  0.01) ( such,  0.00) ( a,  0.06) ( happy,  0.33) (.,  0.17) ( I,  0.03) ( am,  0.17) ( such,  0.03) ( a,  0.33) ( happy,  0.45) ( man,  0.05) (.,  0.18) ( I,  0.11) ( enjoy,  0.15) ( eating,  0.14) ( oranges,  0.18) (.,  0.72) ( I,  0.14) ( strong,  0.23) (ly,  0.22) ( dis,  0.04) (l,  0.03) (ike,  0.05) ( poems,  0.18)

.,! who and to in said was at
[(tensor(14), '.', tensor(0.9993, grad_fn=<UnbindBackward0>)), (tensor(12), ',', tensor(0.0003, grad_fn=<UnbindBackward0>)), (tensor(1), '!', tensor(0.0002, grad_fn=<UnbindBackward0>)), (tensor(658), ' who', tensor(7.9670e-05, grad_fn=<UnbindBackward0>)), (tensor(267), ' and', tensor(6.9575e-05, grad_fn=<UnbindBackward0>)), (tensor(266), ' to', tensor(9.9911e-06, grad_fn=<UnbindBackward0>)), (tensor(313), ' in', tensor(9.4612e-06, grad_fn=<UnbindBackward0>)), (tensor(324), ' said', tensor(6.6216e-06, grad_fn=<UnbindBackward0>)), (tensor(283), ' was', tensor(5.6884e-06, grad_fn=<UnbindBac